In [2]:
from __future__ import division
import keras
import pandas as pd
import math
import os
import mne
from mne import io
from mne.event import define_target_events
import numpy as np
import h5py
import matplotlib.pyplot as plt
from PIL import Image
import glob

Using TensorFlow backend.


In [24]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding, Input

In [104]:
df_train= pd.read_csv('D:/001/arti/separatedEpochs/EEGArtiData_train.csv')
df_test= pd.read_csv("D:/001/arti/separatedEpochs/EEGArtiData_test.csv")

In [105]:
y_train= np.array(pd.get_dummies(df_train['type']))
y_test= np.array(pd.get_dummies(df_test['type']))

In [106]:
print (np.shape(y_train))

(1500, 7)


In [107]:
x_train_names= df_train['signal']
x_train= []
for name in x_train_names:
    raw= mne.read_epochs(name, verbose= False)
    data= (raw.get_data()[0, :, :])
    x_train.append(data.T)
#x_train= np.array(x_train)
#print (len(x_train))

In [108]:
x_train= np.array(x_train)
print (np.shape(x_train))

(1500, 2560, 63)


In [109]:
#values= []
#X_Train= np.array(x_train[0])
#for i in range(0, len(x_train)):
 #   if len(x_train[i])== 1280:
  #      x_train[i]= np.hstack((np.zeros(640), x_train[i], np.zeros(640)))
   # values.append(len(x_train[i]))
    #if i!=0:
     #   X_Train= np.vstack((X_Train, x_train[i]))
#values= np.array(values)
#print (np.unique(values))

In [110]:
#print (X_Train[10])
#print (x_train[10])

In [111]:
X_train= x_train

In [114]:
x_test_names= df_test['signal']
x_test= []
for name in x_test_names:
    raw= mne.read_epochs(name, verbose= False)
    data= raw.get_data()[0, :, :]
    x_test.append(data.T)
x_test= np.array(x_test)
print (len(x_test))

244


In [115]:
X_test= x_test
print (np.shape(x_test))

(244, 2560, 63)


In [116]:
#values= []
#X_Test= np.array(x_test[0])
#for i in range(0, len(x_test)):
 #   if len(x_test[i])== 1280:
  #      x_test[i]= np.hstack((np.zeros(640), x_test[i], np.zeros(640)))
   # values.append(len(x_test[i]))
    #if i!=0:
     #   X_Test= np.vstack((X_Test, x_test[i]))
#values= np.array(values)
#print (np.unique(values))

In [117]:
#print(np.shape(X_Test))
#X_Train= X_Train.reshape((1500, 2560, 1))
#X_Test= X_Test.reshape((244, 2560, 1))
#print(np.shape(X_Test))

In [122]:
input_length= 2560
output_length= 7
model = Sequential()

# Embedding layer
#model.add(Embedding(input_dim=num_words,input_length = training_length,output_dim=100,weights=[embedding_matrix],
              #trainable=False,
              #mask_zero=True))

# Masking layer for pre-trained embeddings
#model.add(Masking(mask_value=0.0))
#model.add(Input(shape= (input_length, )))
model.add(Dense(256, activation= 'tanh', name= "d1"))
# Recurrent layer
model.add(LSTM(32, return_sequences=False, 
               dropout=0.1, recurrent_dropout=0.1, name= 'l1'))

# Fully connected layer
model.add(Dense(256, activation='relu', name='d2'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(output_length, activation='sigmoid', name='d3'))
optimizer= keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# Compile the model
model.compile(optimizer= optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [123]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Create callbacks
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint('D:/001/arti/separatedEpochs/model.h5', save_best_only= True, 
                             save_weights_only=False)]

In [ ]:
history = model.fit(X_Train,  y_train, batch_size=16, epochs=150,callbacks=callbacks,validation_data=(X_Test, y_test))

Train on 1500 samples, validate on 244 samples
Epoch 1/150
1408/1500 [===========================>..] - ETA: 25s - loss: 1.5953 - acc: 0.2592